In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is in

In [2]:
import spacy
from spacy.tokens import DocBin
from datasets import load_dataset
import transformers

In [3]:
dataset = load_dataset("conll2003")
nlp = spacy.blank("en")
doc_bin = DocBin()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [4]:
train_examples = dataset["train"]['tokens']
train_tags = dataset["train"]['ner_tags']

In [5]:
labels = dataset["train"].features["ner_tags"].feature.names

In [6]:
for tokens, ner_tags in zip(dataset["train"]["tokens"], dataset["train"]["ner_tags"]):
    doc = nlp.make_doc(" ".join(tokens))
    ents = []
    start = 0
    for i, (word, tag_id) in enumerate(zip(tokens, ner_tags)):
        tag = labels[tag_id]
        if tag == "O":
            start += len(word) + 1
            continue
        if tag.startswith("B-"):
            ent_start = start
            ent_end = start + len(word)
            ent_label = tag[2:]

            for j in range(i+1, len(tokens)):
                next_tag = labels[ner_tags[j]]
                if next_tag == f"I-{ent_label}":
                    ent_end += len(tokens[j]) + 1
                else:
                    break
            ents.append((ent_start, ent_end - 1, ent_label))
        start += len(word) + 1

    doc.ents = [doc.char_span(start, end + 1, label=label) for start, end, label in ents if doc.char_span(start, end + 1, label=label)]
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

In [7]:
for tokens, ner_tags in zip(dataset["validation"]["tokens"], dataset["validation"]["ner_tags"]):
    doc = nlp.make_doc(" ".join(tokens))
    ents = []
    start = 0
    for i, (word, tag_id) in enumerate(zip(tokens, ner_tags)):
        tag = labels[tag_id]
        if tag == "O":
            start += len(word) + 1
            continue
        if tag.startswith("B-"):
            ent_start = start
            ent_end = start + len(word)
            ent_label = tag[2:]

            for j in range(i+1, len(tokens)):
                next_tag = labels[ner_tags[j]]
                if next_tag == f"I-{ent_label}":
                    ent_end += len(tokens[j]) + 1
                else:
                    break
            ents.append((ent_start, ent_end - 1, ent_label))
        start += len(word) + 1

    doc.ents = [doc.char_span(start, end + 1, label=label) for start, end, label in ents if doc.char_span(start, end + 1, label=label)]
    doc_bin.add(doc)

doc_bin.to_disk("dev.spacy")

In [8]:
!python -m spacy init config config.cfg --lang en --pipeline ner

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [11]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy --training.max_epochs 10


ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     44.28    0.00    0.00    0.00    0.00
  0     200        277.27   2996.40   51.56   53.47   49.79    0.52
  0     400        290.58   2281.65   68.98   68.74   69.22    0.69
  0     600        240.43   1997.41   76.49   77.49   75.51    0.76
  0     800        432.80   1988.48   80.59   81.30   79.89    0.81
  0    1000       1128.01   2315.70   86.51   87.15   85.87    0.87
  1    1200        461.59   2012.91   88.95   89.24   88.65    0.89
  1    1400        478.27   1745.72   90.13   89.93   90.33    0.90
  1    1600        635.11   204

In [22]:
!ls -lh


total 3.9M
-rw-r--r-- 1 root root 2.7K May  4 12:42 config.cfg
-rw-r--r-- 1 root root 2.2M May  4 12:41 dev.spacy
drwxr-xr-x 4 root root 4.0K May  4 12:59 output
drwxr-xr-x 1 root root 4.0K Apr 30 13:37 sample_data
-rw-r--r-- 1 root root 1.8M May  4 12:41 train.spacy


In [25]:
!python -m spacy evaluate output/model-best ./dev.spacy --output metrics.json


ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   97.22 
NER R   97.04 
NER F   97.13 
SPEED   19474 


=============================== NER (per type) ===============================

           P       R       F
ORG    96.99   95.98   96.48
MISC   96.29   95.34   95.82
PER    97.27   97.68   97.48
LOC    97.79   98.18   97.99

✔ Saved results to metrics.json


In [26]:
print(nlp.pipe_labels)


{'tok2vec': [], 'ner': ['LOC', 'MISC', 'ORG', 'PER']}


In [30]:
import spacy

nlp = spacy.load("./output/model-best")
text ="Barack Obama visited Apple headquarters in California during the TechCrunch Disrupt event."
doc = nlp(text)

for ent in doc.ents:
    print(ent.text+"-->", ent.label_)

Barack Obama--> PER
Apple--> ORG
California--> LOC
TechCrunch--> MISC


In [32]:
text = "The Nobel Prize was awarded in Stockholm to Dr. Amina Yusuf for her work with WHO."
doc = nlp(text)

for ent in doc.ents:
    print(ent.text+"-->", ent.label_)

Nobel Prize--> MISC
Stockholm--> LOC
Amina Yusuf--> PER
WHO--> ORG


In [34]:
nlp = spacy.load("en_core_web_sm")
text = "Apple is looking at buying U.K. startup for $1 million"
doc = nlp(text)

for ent in doc.ents:
    print(ent.text+"-->", ent.label_)


Apple--> ORG
U.K.--> GPE
$1 million--> MONEY
